In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sns

In [ ]:
df= pd.read_csv("BostonHousing.csv")
df.head(5)

In [ ]:
selected_cols=['crim','rm','age','dis','rad','medv']
df_filtered= df[selected_cols]
df_filtered.head(5)

#### segregating the dataset into training and testing examples

In [ ]:
X= df_filtered[['crim','rm','age','dis','rad']].values.tolist()
y=df_filtered['medv'].values.tolist()
Xy=list(zip(X,y))
random.shuffle(Xy)
Xy_train=[]; Xy_test=[]
demarc_sample=np.floor(0.7*len(Xy))
for i in range(len(Xy)):
    if(i<demarc_sample):
        Xy_train.append(Xy[i])
    else:
        Xy_test.append(Xy[i])
X_train,y_train=map(list,zip(*Xy_train))
X_test,y_test=map(list,zip(*Xy_test))

In [ ]:
sns.pairplot(data=df_filtered, hue='age')

## standardizing our dataset

In [ ]:
#normalizing X_train
X_train_T=np.array(X_train).T
X_train_std=[]
mean_list=[]
std_dev_list=[]
for features in X_train_T:
    mean=sum(features)/len(features)
    mean_list.append(mean)
    std_dev=(sum([(x-mean)**2 for x in features])/len(features))**0.5
    std_dev_list.append(std_dev)
    x_std=[(x-mean)/std_dev for  x in features]
    X_train_std.append(x_std)
X_train=np.array(X_train_std).T

#normalizing X_test
X_test_T=np.array(X_test).T
X_test_std=[]
for j,features in enumerate(X_test_T):
    x_std=[(x-mean_list[j])/std_dev_list[j] for x in features]
    X_test_std.append(x_std)
X_test=np.array(X_test_std).T

## single batch gradient descent 

In [ ]:
loss_history=[]

class LinearRegression:
    def __init__(self,X_train):
        self.w=[random.uniform(-1,1) for _ in range(len(X_train[0]))]
        self.b=random.uniform(-1,1)
        
    def loss_function(self,X_train,y_train):
        SE=0
        for i in range(len(X_train)):
            y_pred=sum((wi*xi for wi,xi in zip(self.w,X_train[i])),self.b) 
            y_tg=y_train[i]
            SE+=(y_pred-y_tg)**2
        RMSE=(SE/len(X_train))**0.5
        return RMSE
        
    def gradient_descent(self,X_train,y_train,alpha):
        error=0
        w_grad=[0 for _ in range(len(self.w))]
        b_grad=0
        for i in range(len(X_train)):
            y_pred=sum((wi*xi for wi,xi in zip(self.w,X_train[i])),self.b) 
            y_tg=y_train[i]
            error=(y_pred-y_tg)
            for j in range(len(self.w)):
                w_grad[j]+=error*X_train[i][j]
            b_grad+=error
        for k in range(len(self.w)):
            self.w[k]+=(-2/len(X_train))*alpha*w_grad[k]
        self.b+=(-2/len(X_train))*alpha*b_grad
        return f' w={self.w} , b={self.b}'
        
    def predict(self,X_test):
        prediction=[]
        for i in range(len(X_test)):
            prediction.append(sum((wi*xi for wi,xi in zip(self.w,X_test[i])),self.b))
        return prediction

    def R_squared(self,X_test,y_test):
        y_mean=sum(y_test)/len(y_test)
        TSS=sum([(y-y_mean)**2 for y in y_test])
        y_pred=[]
        for i in range(len(X_test)):
            y_pred.append(sum((wi*xi for wi,xi in zip(self.w,X_test[i])),self.b))
        RSS=sum([(y1-y2)**2 for y1,y2 in zip(y_pred,y_test)])
        return 1-(RSS/TSS)

epochs=1000

model=LinearRegression(X_train)
for i in range(epochs):
    model.gradient_descent(X_train,y_train,alpha=0.01)
    print(model.loss_function(X_train,y_train))
    loss_history.append(model.loss_function(X_train,y_train))         

 ##### the model's RMSE would not converge to zero because of the noise in the dataset

### plotting the RMSE for each epoch

In [ ]:
sns.lineplot(x=range(epochs),y=loss_history, color='red', linewidth=2)
plt.xlabel("epochs")
plt.ylabel("RMSE")

## testing our model on test dataset

In [ ]:
y_pred=[float(x) for x in model.predict(X_test)]
print(*y_pred, sep='\n')

In [ ]:
sns.scatterplot(x=y_test,y=y_pred)
sns.lineplot(x=y_test,y=y_test, linestyle='--',color='red')
plt.xlabel("actual median value"); plt.ylabel("predicted median value")
plt.title("Accuracy of my model")

In [ ]:
print(model.R_squared(X_test,y_test))